In [ ]:
#S-Box Layer
s_box=[0xC,0x5,0x6,0xB,0x9,0x0,0xA,0xD,0x3,0xE,0xF,0x8,0x4,0x7,0x1,0x2]

# Rotate left: 0b1001 --> 0b0011
rol = lambda val, r_bits, max_bits: \
    (val << r_bits%max_bits) & (2**max_bits-1) | \
    ((val & (2**max_bits-1)) >> (max_bits-(r_bits%max_bits)))

# Rotate right: 0b1001 --> 0b1100
ror = lambda val, r_bits, max_bits: \
    ((val & (2**max_bits-1)) >> r_bits%max_bits) | \
    (val << (max_bits-(r_bits%max_bits)) & (2**max_bits-1))

# s-present는 Present의 키스케줄링 동일. 오른쪽 4n bit로만 구성
def genRoundKeys(key, rounds=32):
    roundkeys = []
    for i in range(1,rounds+1): # (K1 ... K32)
        # rawkey: used in comments to show what happens at bitlevel
        # rawKey[0:64]
        roundkeys.append(key >>16)
        #1. Shift
        #rawKey[19:len(rawKey)]+rawKey[0:19]
        key = ((key & (2**19-1)) << 61) + (key >> 19)
        #2. SBox
        #rawKey[76:80] = S(rawKey[76:80])
        key = (s_box[key >> 76] << 76)+(key & (2**76-1))
        #3. Salt
        #rawKey[15:20] ^ i
        key ^= i << 15

    return roundkeys

#############################################

import numpy as np

def Sbox(x):
    temp_x = x & 0b1111
    temp_y = x >> 4

    table = [0xc, 0x5, 0x6, 0xb, 0x9, 0x00, 0xa, 0xd, 0x3, 0xe, 0xf, 0x8, 0x4, 0x7, 0x1, 0x2]
    
    temp_x = table[temp_x]
    temp_y = table[temp_y]

    temp = [0,0,0,0]
    temp[0] = (temp_y & 0b0001)
    temp[1] = (temp_y & 0b0010) >> 1
    temp[2] = (temp_y & 0b0100) >> 2
    temp[3] = (temp_y & 0b1000) >> 3

    result = temp_x + temp[0] * 16 +  temp[1] * 32 + temp[2] * 64 + temp[3] * 128

    return result

def Permutation(x):

    # 73625140
    temp = [0,0,0,0,0,0,0,0]
    temp[0] = x & 0b0001
    temp[1] = (x & 0b0010) >> 1
    temp[2] = (x & 0b0100) >> 2
    temp[3] = (x & 0b1000) >> 3

    temp[4] = (x & 0b00010000) >> 4
    temp[5] = (x & 0b00100000) >> 5
    temp[6] = (x & 0b01000000) >> 6
    temp[7] = (x & 0b10000000) >> 7

    result = 0
    result = result + temp[0]
    result = result + temp[4] * 2
    result = result + temp[1] * 4
    result = result + temp[5] * 8
    result = result + temp[2] * 16
    result = result + temp[6] * 32
    result = result + temp[3] * 64
    result = result + temp[7] * 128

    return result

def Present(pt, key, rounds, n):
    roundkeys = genRoundKeys(key, rounds)
    s_rnd = []
    
    for rnd in range(len(roundkeys)):         
        short = int(bin(roundkeys[rnd])[-4*n:],2)
        s_rnd.append(short)
    #print(s_rnd)
    #for i in s_rnd:
    #  print(hex(i))
    
    for i in range(rounds-1):
        pt = pt ^ s_rnd[i] #key
        pt = Sbox(pt)
        pt = Permutation(pt)
    ct = pt ^ s_rnd[-1]

    return ct